In [1]:
# for PyCharm
import os

current_directory = os.getcwd()
project_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
os.chdir(project_directory)

In [2]:
import pandas as pd
from pathlib import Path

from src.data.merger import Merger

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
datasets = Merger().get_formatted()
Merger.check_extra_symbols(datasets, column="brand")

Superkicks


Sneakerbaas


Footshop


Kickscrew


superkicks dict_keys([])
sneakerbaas dict_keys([])
footshop dict_keys(['.', '®', "'", '*'])
kickscrew dict_keys(['.'])


In [4]:
datasets["footshop"]

,brand,title,color,slug,brand_slug,pricecurrency,price,collection_name,url,images_path,website,title_without_color,title_merge
0,Under Armour,Under Armour HOVR Machina Breeze,"[blue surf, white, orange blast]",under-armour-hovr-machina-breeze,under-armour,EUR,113.72,5-mens-shoes,https://www.footshop.eu/en/mens-shoes/266188-u...,data/raw/images/footshop/5-mens-shoes/under-ar...,footshop,Under Armour HOVR Machina Breeze,hovr machina breeze
1,Asics,Asics x Andersson Bell Gel-Sonoma 15-50,"[olive oil, dark brown]",asics-x-andersson-bell-gelsonoma-1550,asics,EUR,155.51,5-mens-shoes,https://www.footshop.eu/en/mens-shoes/264451-a...,data/raw/images/footshop/5-mens-shoes/asics/as...,footshop,Asics x Andersson Bell Gel-Sonoma 15-50,bell gel sonoma 15 50
2,adidas Originals,adidas Crazy Iiinfinity,"[silver metallic, core black, cream white]",adidas-crazy-iiinfinity,adidas-originals,EUR,160.00,5-mens-shoes,https://www.footshop.eu/en/mens-shoes/291355-a...,data/raw/images/footshop/5-mens-shoes/adidas-o...,footshop,adidas Crazy Iiinfinity,crazy iiinfinity
3,adidas Originals,adidas NMD S1,"[core black, core black, alter navy]",adidas-nmd-s1,adidas-originals,EUR,135.00,5-mens-shoes,https://www.footshop.eu/en/mens-shoes/232945-a...,data/raw/images/footshop/5-mens-shoes/adidas-o...,footshop,adidas NMD S1,nmd s1
4,adidas Originals,adidas x Wales Bonner Pony Tonal Samba,"[core black, core white, core white]",adidas-x-wales-bonner-pony-tonal-samba,adidas-originals,EUR,200.00,5-mens-shoes,https://www.footshop.eu/en/mens-shoes/291166-a...,data/raw/images/footshop/5-mens-shoes/adidas-o...,footshop,adidas x Wales Bonner Pony Tonal Samba,pony tonal samba
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6418,New Balance,New Balance CT302,[all black],new-balance-ct302,new-balance,EUR,114.95,55-kids-sneakers-and-shoes,https://www.footshop.eu/en/kids-sneakers-and-s...,data/raw/images/footshop/55-kids-sneakers-and-...,footshop,New Balance CT302,ct302
6419,Nike,Nike Court Borough Mid 2,"[university red, black-white]",nike-court-borough-mid-2,nike,EUR,74.95,55-kids-sneakers-and-shoes,https://www.footshop.eu/en/kids-sneakers-and-s...,data/raw/images/footshop/55-kids-sneakers-and-...,footshop,Nike Court Borough Mid 2,court borough mid 2
6420,Vans,Vans Old Skool Kids,"[black, black]",vans-old-skool-kids,vans,EUR,59.95,55-kids-sneakers-and-shoes,https://www.footshop.eu/en/kids-sneakers-and-s...,data/raw/images/footshop/55-kids-sneakers-and-...,footshop,Vans Old Skool Kids,old skool kids
6421,Vans,Vans Kids Old Skool,"[navy, true white]",vans-kids-old-skool,vans,EUR,59.95,55-kids-sneakers-and-shoes,https://www.footshop.eu/en/kids-sneakers-and-s...,data/raw/images/footshop/55-kids-sneakers-and-...,footshop,Vans Kids Old Skool,kids old skool


In [5]:
# apply deletion of colors

In [6]:
for key, value in datasets.items():
    print(value.shape)

(1090, 13)
(999, 14)
(4852, 13)
(4900, 10)


In [7]:
# same columns
same_columns = set.intersection(*[set(datasets[key].columns) for key in datasets])
same_columns
# no colors and collection-name

{'brand',
 'color',
 'images_path',
 'price',
 'pricecurrency',
 'title',
 'title_merge',
 'title_without_color',
 'url',
 'website'}

In [8]:
merged_datasets = pd.concat(list(datasets.values()), ignore_index=True)
merged_datasets

,brand,title,price,slug,brand_slug,collection_name,url,images_path,website,pricecurrency,title_without_color,color,title_merge,description_color
0,Adidas Originals,CAMPUS 00S 'CLOUD WHITE/BLUE',11999.0,campus-00s-cloud-whiteblue,adidas-originals,men-sneakers,https://www.superkicks.in/products/campus-00s-...,data/raw_new/images/superkicks/men-sneakers/ad...,superkicks,INR,CAMPUS 00S,CLOUD WHITE/BLUE,campus 00s,NaN
1,Converse,WMN'S RUN STAR MOTION CX PLATFORM TORTOISE 'KH...,9999.0,wmns-run-star-motion-cx-platform-tortoise-khak...,converse,men-sneakers,https://www.superkicks.in/products/wmns-run-st...,data/raw_new/images/superkicks/men-sneakers/co...,superkicks,INR,WMN'S RUN STAR MOTION CX PLATFORM TORTOISE,KHAKI/OFF WHITE,wmns run star motion cx platform tortoise,NaN
2,Nike,KEVIN DURANT KD16 EP 'BLACK/BLACK-DK SMOKE GRE...,13995.0,kevin-durant-kd16-ep-blackblackdk-smoke-greyco...,nike,men-sneakers,https://www.superkicks.in/products/kevin-duran...,data/raw_new/images/superkicks/men-sneakers/ni...,superkicks,INR,KEVIN DURANT KD16 EP,BLACK/BLACK-DK SMOKE GREY-COCONUT MILK,kevin durant kd16 ep,NaN
3,Puma,STAPLE B SUEDE 'DARK CHOCOLATE-RHUBARB',9999.0,staple-b-suede-dark-chocolaterhubarb,puma,men-sneakers,https://www.superkicks.in/products/staple-b-su...,data/raw_new/images/superkicks/men-sneakers/pu...,superkicks,INR,STAPLE B SUEDE,DARK CHOCOLATE-RHUBARB,staple b suede,NaN
4,Nike,DUNK LOW RETRO PREMIUM 'CACAO WOW/COMET BLUE-M...,9695.0,dunk-low-retro-premium-cacao-wowcomet-bluemars...,nike,men-sneakers,https://www.superkicks.in/products/dunk-low-re...,data/raw_new/images/superkicks/men-sneakers/ni...,superkicks,INR,DUNK LOW RETRO PREMIUM,CACAO WOW/COMET BLUE-MARS STONE-PHANTOM,dunk low retro premium,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11836,adidas,Adidas Ultraboost 1.0 Shoes 'Valentines Day',123.0,NaN,NaN,NaN,https://www.kickscrew.com/products/adidas-ultr...,[data/raw/images/kickscrew/adidas/adidas-ultra...,kickscrew,USD,Adidas Ultraboost 1.0 Shoes,Valentines Day,ultraboost 10 shoes,NaN
11837,adidas,Adidas Originals Ozweego Shoes 'Magic Beige Cl...,102.0,NaN,NaN,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,USD,Adidas Originals Ozweego Shoes,Magic Beige Clay Strata,ozweego shoes,NaN
11838,adidas,Adidas Originals Adilette Slides 'Off White Br...,56.0,NaN,NaN,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,USD,Adidas Originals Adilette Slides,Off White Bright Orange,adilette slides,NaN
11839,adidas,Adidas Originals Forum Low Shoes 'White Grey Red',84.0,NaN,NaN,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,USD,Adidas Originals Forum Low Shoes,White Grey Red,forum low shoes,NaN


In [9]:
# group by по названиям, если все названия будут одинаковые, то все дублирующие кроссы сложатся в одно.
test = merged_datasets.groupby("title_merge").agg({
    "brand": list,
    "collection_name": list,
    "color": list,
    "images_path": list,
    "price": list,
    "pricecurrency": list,
    "url": list,
    "website": list
}).reset_index().sort_values(by="title_merge")
test

,title_merge,brand,collection_name,color,images_path,price,pricecurrency,url,website
0,01 low,"[Autry, Autry]","[category-women, category-men]","[White, Mustard]",[data/raw_new/images/sneakerbaas/category-wome...,"[129.99, 149.99]","[EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas]"
1,01 low m,"[Autry, Autry, Autry]","[category-men, category-men, category-men]","[BLACK, BI VIOLET, BI EDEN]",[data/raw_new/images/sneakerbaas/category-men/...,"[149.99, 149.99, 149.99]","[EUR, EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas, sneakerbaas]"
2,01 low man,[Autry],[category-men],[LEAT/LEAT RED],[data/raw_new/images/sneakerbaas/category-men/...,[149.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
3,01 low w,"[Autry, Autry, Autry, Autry]","[category-women, category-women, category-wome...","[BI POW, WHT/CORAL, SALTWATER, BI IVORY]",[data/raw_new/images/sneakerbaas/category-wome...,"[149.99, 149.99, 149.99, 149.99]","[EUR, EUR, EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas, sneakerbaas, sneake..."
4,01 low wom,[Autry],[category-women],[WHT/PETROL],[data/raw_new/images/sneakerbaas/category-wome...,[149.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
...,...,...,...,...,...,...,...,...,...
4937,zx 8000,"[adidas Originals, adidas Originals]","[5-mens-shoes, 5-mens-shoes]","[[sky tint, core white], [ice mint, trace pink...",[data/raw/images/footshop/5-mens-shoes/adidas-...,"[169.96, 149.97]","[EUR, EUR]",[https://www.footshop.eu/en/mens-shoes/257950-...,"[footshop, footshop]"
4938,zx 8000 minimalist,[adidas Originals],[5-mens-shoes],"[[ftw white, grey two, ftw white]]",[data/raw/images/footshop/5-mens-shoes/adidas-...,[82.5],[EUR],[https://www.footshop.eu/en/mens-shoes/102730-...,[footshop]
4939,zx 9020 w,[adidas Originals],[6-womens-shoes],"[[cloud white, core white, glory mint]]",[data/raw/images/footshop/6-womens-shoes/adida...,[83.26],[EUR],[https://www.footshop.eu/en/womens-shoes/25753...,[footshop]
4940,zx flux slip,[adidas],[nan],[Solar Yellow Red],[[data/raw/images/kickscrew/adidas/adidas-orig...,[212.0],[USD],[https://www.kickscrew.com/products/adidas-ori...,[kickscrew]


In [10]:
# sizes
print(merged_datasets.shape)
print(test.shape)

(11841, 14)
(4942, 9)


In [11]:
test.to_csv("data/raw/metadata/dataset.csv", index=False)

In [24]:
# import re
# 
# # delete any brand from title:
# all_brands = set(merged_datasets["brand"].unique())
# all_brands = [re.escape(brand) for stripped_brand in all_brands for brand in stripped_brand.split(" ")]
# all_brands = "|".join(all_brands)
# all_brands

"Diadora|Diesel|Adidas|Palm|Angels|Crocs|Under|Armour|Etonic|Alexander|McQueen|Fendi|Onitsuka|Tiger|CASIO|Clarks|Helly|Hansen|adidas|originals|Skechers|GUCCI|McQ|HOKA®|New|Balance|Hi\\-Tec|UGG|361|Degrees|Salomon|PUMA|Sportstyle|Vans|Vault|Salomon|Advanced|HERON|PRESTON|RIPNDIP|Y\\-3|Mitchell|\\&|Ness|Saucony|Originals|On|Converse|Puma|Sportstyle|adidas|Toms|HOKA|ONE|ONE|Fjällräven|adidas|Originals|Levi's®|Stepney|Workers|Club|Tommy|Hilfiger|Suicoke|Nike|Reebok|Classics|SALOMON|Dr\\.|Martens|Timberland|Fear|of|God|Essentials|Ralph|Lauren|Karhu|KARHU|Asics|MISBHV|Balenciaga|Mizuno|MARKET|DIOR|Rigorer|FILA|Comme|des|Garçons|PLAY|ETONIC|PLEASURES|Reebok|Warrior|RAF|SIMONS|Adidas|Originals|LACOSTE|Saucony|Sergio|Tacchini|nike|Nike|Skateboarding|Filling|Pieces|Ellesse|Maison|MIHARA|YASUHIRO|Fila|Camper|Kangaroos|On|Running|Hi\\-tec|Moon|Boot|Clarks|Originals|Vans|Jordan|adidas|Performance|Columbia|VANS|KangaROOS|Merrell|Just|Don|A\\-COLD\\-WALL\\*|ANTA|PRADA|Ambush|Birkenstock|Represent|Foo

In [29]:
# # delete any brand name from title:
# test["title_no_brand"] = test["title_merge"].str.replace(all_brands, "", regex=True)
# # diff between title and title without brand
# test["title_diff"] = test["title_merge"].str.len() - test["title_no_brand"].str.len()
# test["title_diff"].value_counts()

title_diff
0     3174
4     1075
6      647
7      116
15      84
2       79
3       34
9        9
8        7
10       2
18       1
12       1
Name: count, dtype: int64

In [32]:
# # sample of changed titles:
# test[test["title_no_brand"] != test['title_merge']].sample(10)[["title_merge", 'title_no_brand']]

,title_merge,title_no_brand
496,adidas supercourt,supercourt
2629,nike concepts x air max 1 sp,concepts x air max 1 sp
2899,nike revolution 4,revolution 4
2458,nike air max 90 qs,air max 90 qs
407,adidas pokemon x advantage j,pokemon x advantage j
3489,ps nike space jam x lebron 19,ps space jam x lebron 19
135,adidas country og,country og
395,adidas ozweego celox,ozweego celox
248,adidas james bond 007 x ultraboost summerrdy,james bond 007 x ultraboost summerrdy
2640,nike court borough mid winter,court borough mid winter


In [39]:
# # cleanup
# test.columns

Index(['title_merge', 'brand', 'collection_name', 'color', 'images_path',
       'price', 'pricecurrency', 'url', 'website', 'title_diff',
       'title_no_brand'],
      dtype='object')

In [40]:
# test['title'] = test['title_no_brand']
# test.drop(columns=["title_merge", "title_no_brand", "title_diff"], inplace=True)

In [41]:
# # finally, save:
# test.to_csv("data/raw/metadata/dataset.csv", index=False)

## Seara heavymetal experiments:

In [12]:
# !pip install thefuzz -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
# from thefuzz import fuzz
# 
# 
# def similarity(string1, string2):
#     return fuzz.token_sort_ratio(string1, string2)
# 
# 
# title = "vans - ua og authentic lx yellow"
# 
# similarities = []
# for text in titles:
#     similarities.append((text, similarity(title, text)))
# 
# similarities.sort(key=lambda x: x[1])
# similarities[-10:]

[('vans ua og authentic lx vault checkerboard', 69),
 ('vans vault authentic one piece vlt lx', 69),
 ('vans vault og authentic frayed lx', 70),
 ('vans vault og authentic lx canvas', 70),
 ('authentic los vans', 71),
 ('vans og authentic lx stressed', 71),
 ('vans ua authentic vr3 pw lx', 77),
 ('vans ua og authentic lx suede', 78),
 ('vans vault og authentic lx', 79),
 ('vans og authentic lx', 80)]

In [18]:
# эта тема лучше всего находит похожие

In [14]:
# test.to_csv("temp.csv")